# 1141_社會資料分析 作業四
學生：劉晏成

以「2018台北大學社會系三鶯調查」（brilliant_working.sav）檔案回答以下問題：
1. 以變異數分析方法（包括整體檢定與事後比較），檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。
2. 以教育年數（eduy）作為共變量，再次檢視檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/brilliant_working.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

## Q1: 以變異數分析方法（包括整體檢定與事後比較），檢視不同年齡分組者（agegroup）的收入（inc）是否有顯著差異。

In [3]:
df_1 = df[['agegroup', 'inc']].copy()

首先，先確認不同年齡組別分布

In [4]:
df_1['agegroup'].value_counts()

agegroup
1.0    336
2.0    327
3.0    205
Name: count, dtype: int64

確認共三組（1, 2, 3）

確認缺漏值狀況

In [5]:
from statistic.missing_value import check_missing_value

check_missing_value(df_1)

,# of Missing Values
agegroup,0
inc,4


發現有缺漏值，預處理inc 為缺漏值的資料

In [6]:
df_1_cleared = df_1.dropna()
check_missing_value(df_1_cleared)

,# of Missing Values
agegroup,0
inc,0


In [7]:
from scipy.stats import f_oneway

groups = {name: group['inc'].values for name, group in df_1_cleared.groupby('agegroup')}

f_stat, anova_p = f_oneway(groups[1], groups[2], groups[3])
pd.DataFrame({ 'Values': [f"{f_stat:.2f}", anova_p] }, index = ['F Stat', 'P Value'])

,Values
F Stat,14.44
P Value,0.000001


依據P Value 結果顯示三組間存在差異

接下來執行Levene's test 來判斷均值性狀況，以執行事後比較

In [8]:
from scipy.stats import levene


levene_stat, levene_p = levene(*groups.values())
pd.DataFrame({ 'Values': [f"{levene_stat:.2f}", levene_p] }, index = ['Levene Stat', 'P Value'])

,Values
Levene Stat,6.80
P Value,0.001176


結果顯示P Value 遠小於0.05：非均值。在此選擇Dunnett's T3 test 來做事後比較

In [11]:
import scikit_posthocs as sp
from scipy.stats import tukey_hsd


if levene_p > 0.05:
    # tukey_hsd can use the dictionary directly (scipy >= 1.11)
    tukey_result = tukey_hsd(*groups.values())
    # Print with labels
    group_names = list(groups.keys())
    print(f"Groups: {group_names}")
    print(tukey_result)
else:
    # scikit_posthocs works directly with dataframe
    dunnett_t3 = sp.posthoc_dunnett(df_1_cleared, val_col="inc", group_col="agegroup", control=1.0)
    print(dunnett_t3)

          1.0       3.0       2.0
1.0       1.0  0.049529  0.001166
3.0  0.049529       NaN       NaN
2.0  0.001166       NaN       NaN
